## LLM chat completions

In [1]:
from openai import OpenAI
import json
from rich import print

import dotenv
dotenv.load_dotenv()

False

In [2]:
client = OpenAI()

In [3]:
messages = [{"role": "user", "content": "What's the weather like in London?"}]
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)

In [4]:
print(response)

ChatCompletion(
    id='chatcmpl-9W1ds9NarR8iLAlLgWlEjkJvMF4VY',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content="I'm sorry, but I can't provide real-time information, including current weather 
conditions. I recommend checking a reliable weather website or app like the BBC Weather, Met Office, or any weather
service of your choice to get the most up-to-date information about London's weather.",
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1717419836,
    model='gpt-4o-2024-05-13',
    object='chat.completion',
    system_fingerprint='fp_319be4768e',
    usage=CompletionUsage(completion_tokens=52, prompt_tokens=14, total_tokens=66)
)

In [5]:
def get_current_weather(location):
    """Get the current weather in a given city"""
    if "london" in location.lower():
        return json.dumps({"temperature": "20 C"})
    elif "san francisco" in location.lower():
        return json.dumps({"temperature": "15 C"})
    elif "paris" in location.lower():
        return json.dumps({"temperature": "22 C"})
    else:
        return json.dumps({"temperature": "unknown"})



messages = [{"role": "user", "content": "What's the weather like in London?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco",
                    },
                },
                "required": ["location"],
            },
        },
    }
]

In [6]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(response)

ChatCompletion(
    id='chatcmpl-9W1dutKQMajxDPsAlRbGgcwUNtNrD',
    choices=[
        Choice(
            finish_reason='tool_calls',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                role='assistant',
                function_call=None,
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_W6I2mTZaVI3PTj6NhZ6LE9aE',
                        function=Function(arguments='{"location":"London"}', name='get_current_weather'),
                        type='function'
                    )
                ]
            )
        )
    ],
    created=1717419838,
    model='gpt-4o-2024-05-13',
    object='chat.completion',
    system_fingerprint='fp_319be4768e',
    usage=CompletionUsage(completion_tokens=15, prompt_tokens=66, total_tokens=81)
)

## Agents & Tool use

In [7]:
from agents.llms import OpenAIChatCompletion

from agents.tools import get_current_weather
from agents.tool_executor import need_tool_use

In [8]:
llm = OpenAIChatCompletion()
llm.bind_tools([get_current_weather])

In [9]:
messages = [
    {"role": "user", "content": "how is the weather in London today?"}
]

output = llm.chat_completion(messages)
print(output)

ChatCompletion(
    id='chatcmpl-9W1dwQZwSBnoFWP1zQMZHeLXxvn6N',
    choices=[
        Choice(
            finish_reason='tool_calls',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                role='assistant',
                function_call=None,
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_7TijvD5Fwx1hd2fSwuWFzEZi',
                        function=Function(arguments='{"city":"London"}', name='get_current_weather'),
                        type='function'
                    )
                ]
            )
        )
    ],
    created=1717419840,
    model='gpt-4o-2024-05-13',
    object='chat.completion',
    system_fingerprint='fp_319be4768e',
    usage=CompletionUsage(completion_tokens=15, prompt_tokens=83, total_tokens=98)
)

In [10]:
if need_tool_use(output):
    print("Using weather tool")
    tool_results = llm.run_tools(output)
    print(tool_results)
    tool_results[0]["role"] = "assistant"

    updated_messages = messages + tool_results
    updated_messages = updated_messages + [
        {"role": "user", "content": "Think step by step and answer my question based on the above context."}
    ]
    output = llm.chat_completion(updated_messages)

print(output.choices[0].message.content)

Using weather tool

[
    {
        'tool_call_id': 'call_7TijvD5Fwx1hd2fSwuWFzEZi',
        'role': 'tool',
        'name': 'get_current_weather',
        'content': '{"FeelsLikeC": "16", "FeelsLikeF": "61", "cloudcover": "100", "humidity": "77", 
"localObsDateTime": "2024-06-03 09:56 AM", "observation_time": "08:56 AM", "precipInches": "0.0", "precipMM": 
"0.0", "pressure": "1022", "pressureInches": "30", "temp_C": "16", "temp_F": "61", "uvIndex": "4", "visibility": 
"10", "visibilityMiles": "6", "weatherCode": "122", "weatherDesc": [{"value": "Overcast"}], "weatherIconUrl": 
[{"value": ""}], "winddir16Point": "NW", "winddirDegree": "310", "windspeedKmph": "7", "windspeedMiles": "4"}'
    }
]

Let's break down the weather details for London today:

1. **Temperature**:
   - The temperature is **16°C** (61°F).

2. **Feels Like**:
   - Although the actual temperature is 16°C, it feels like 16°C (61°F).

3. **Cloud Cover**:
   - The cloud cover is at 100%, indicating that it is very cloudy.

4. **Humidity**:
   - The humidity level is 77%, which is relatively high.

5. **Precipitation**:
   - There is no recorded precipitation (0.0 mm or 0.0 inches).

6. **Pressure**:
   - The atmospheric pressure is **1022 mbar** (30 inches).

7. **UV Index**:
   - The UV Index is 4, suggesting moderate UV exposure.

8. **Visibility**:
   - Visibility is at 10 km (6 miles), which is typical.

9. **Weather Description**:
   - The weather is described as "Overcast," meaning the sky is covered with clouds.

10. **Wind**:
    - The wind is coming from the northwest (NW) at a speed of 7 km/h (4 mph).

**Summary**:
The weather in London today is overcast with a temperature of 16°C (61°F), high humidity at 77%, no precipitation, 
moderate UV Index, and light winds from the northwest.

## Langchain tools

In [11]:
from agents.tool_executor import ToolRegistry, need_tool_use
from agents.tools import get_current_weather, wikipedia_search, google_search, image_inspector
import json
from langchain.tools import tool

In [40]:
@tool
def get_current_weather(city: str) -> str:
    """Get the current weather for a given city.

    Args:
      city (str): The city to fetch weather for.

    Returns:
      str: current weather condition, or None if an error occurs.
    """
    try:
        data = json.dumps({"city": city, "temperature": "12 Celsius", "humidity": "77", "pressure": "1022"})
        return data
    except Exception as e:
        logger.exception(e)
        error_message = f"Error fetching current weather for {city}: {e}"
        return error_message

In [41]:
registry = ToolRegistry()
registry.register_tool(get_current_weather)
print(registry.openai_tools[0])

{
    'type': 'function',
    'function': {
        'name': 'get_current_weather',
        'description': 'Get the current weather for a given city.\n\n    Args:\n      city (str): The city to fetch
weather for.\n\n    Returns:\n      str: current weather condition, or None if an error occurs.',
        'parameters': {'type': 'object', 'properties': {'city': {'type': 'string'}}, 'required': ['city']}
    }
}

In [42]:
llm = OpenAIChatCompletion()

In [15]:
messages = [
    {"role": "user", "content": "how is the weather in London today?"}
]

output = llm.chat_completion(messages)
print(output)

ChatCompletion(
    id='chatcmpl-9W1e3d9X9RynV3kcHEsSSpvUXJr8r',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content='I cannot provide real-time information as my training data only goes up until October 
2021, and I cannot access the internet for live data. To find out the current weather in London, I recommend 
checking a reliable weather website or app like the BBC Weather, the Met Office, or any other trusted weather 
forecasting service.',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1717419847,
    model='gpt-4o-2024-05-13',
    object='chat.completion',
    system_fingerprint='fp_319be4768e',
    usage=CompletionUsage(completion_tokens=64, prompt_tokens=15, total_tokens=79)
)

In [43]:
llm.bind_tools([get_current_weather])

In [44]:
messages = [
    {"role": "user", "content": "how is the weather in London today?"}
]

output = llm.chat_completion(messages)
print(output)

ChatCompletion(
    id='chatcmpl-9W1ixpciQ06L8LZNCpdrdvPcjjgnJ',
    choices=[
        Choice(
            finish_reason='tool_calls',
            index=0,
            logprobs=None,
            message=ChatCompletionMessage(
                content=None,
                role='assistant',
                function_call=None,
                tool_calls=[
                    ChatCompletionMessageToolCall(
                        id='call_maILfBMp3VlqnI750KD14PFE',
                        function=Function(arguments='{"city":"London"}', name='get_current_weather'),
                        type='function'
                    )
                ]
            )
        )
    ],
    created=1717420151,
    model='gpt-4o-2024-05-13',
    object='chat.completion',
    system_fingerprint='fp_319be4768e',
    usage=CompletionUsage(completion_tokens=15, prompt_tokens=83, total_tokens=98)
)

In [45]:
if need_tool_use(output):
    print("Using weather tool")
    tool_results = llm.run_tools(output)
    print(tool_results)
    tool_results[0]["role"] = "system"

    updated_messages = messages + tool_results
    updated_messages = updated_messages + [
        {"role": "user", "content": "think step by step and answer my question based on the above context."}
        ]
    output = llm.chat_completion(updated_messages)

print(output.choices[0].message.content)

Using weather tool

[
    {
        'tool_call_id': 'call_maILfBMp3VlqnI750KD14PFE',
        'role': 'tool',
        'name': 'get_current_weather',
        'content': '{"city": "London", "temperature": "12 Celsius", "humidity": "77", "pressure": "1022"}'
    }
]

Let's break down the information provided step by step to answer your question about the weather in London today:

1. **City**: London
2. **Temperature**: 12 degrees Celsius
3. **Humidity**: 77%
4. **Pressure**: 1022 hPa (hectopascals)

Based on these data points:
- The temperature is relatively cool at 12 degrees Celsius.
- The humidity level is moderately high at 77%.
- The atmospheric pressure of 1022 hPa is considered normal.

So, the weather in London today is cool with quite a bit of humidity and normal atmospheric pressure. This suggests
that it might feel a bit chilly and possibly damp, but the pressure indicates stable weather conditions.